In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient  # 👈 Ajouté ici

# Config MLflow
mlflow.set_tracking_uri("http://localhost:5001")
mlflow.set_experiment("LogisticRegression-Loan")

output_dir = "/tmp/mlflow_outputs"
os.makedirs(output_dir, exist_ok=True)

# Chargement des données
df = pd.read_csv("Loan_Data.csv")

X = df.drop(['customer_id', 'default'], axis=1)  
y = df['default']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

param_grid = [
    {"C": 1.0, "solver": "liblinear"},
    {"C": 0.1, "solver": "liblinear"},
    {"C": 10.0, "solver": "lbfgs", "max_iter": 1000},
]

# Boucle d'entraînement
for params in param_grid:
    with mlflow.start_run() as run:  # 👈 récupération du run pour avoir run_id
        print(f"\n➡️ Training LogisticRegression avec params: {params}")
        
        # Modèle
        model = LogisticRegression(**params)
        model.fit(X_train, y_train)
        
        # Prédiction
        y_pred = model.predict(X_test)
        
        # Métriques
        acc = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        
        # Log hyperparamètres & métriques
        mlflow.log_params(params)
        mlflow.log_metric("Accuracy", acc)
        mlflow.log_metric("Precision", precision)
        mlflow.log_metric("Recall", recall)
        
        # Log du modèle
        mlflow.sklearn.log_model(model, "model")
        
        # Log matrice de confusion
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(5,4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.xlabel("Prédictions")
        plt.ylabel("Vraies valeurs")
        plt.title("Matrice de confusion")
        plt.tight_layout()
        
        image_path = os.path.join(output_dir, "confusion_matrix.png")
        plt.savefig(image_path)
        plt.close()
        mlflow.log_artifact(image_path)
        os.remove(image_path)
        
        ### Enregistrement dans le Model Registry ###
        model_name = "Loan_Model_v1"
        client = MlflowClient()
        
        # Créer le Registered Model si pas encore créé
        try:
            client.create_registered_model(model_name)
        except Exception as e:
            print(f"Model déjà enregistré : {e}")
        
        # Créer une nouvelle version
        model_src = f"{run.info.artifact_uri}/model"
        client.create_model_version(
            name=model_name,
            source=model_src,
            run_id=run.info.run_id
        )
        
print("\ Tous les runs sont loggés ET enregistrés dans le Model Registry ! Vérifie sur : http://localhost:5001")



<>:95: SyntaxWarning: invalid escape sequence '\ '
<>:95: SyntaxWarning: invalid escape sequence '\ '
/var/folders/7k/tt6zbh8s3hs02rgg21dbldzr0000gn/T/ipykernel_28301/3936058750.py:95: SyntaxWarning: invalid escape sequence '\ '
  print("\ Tous les runs sont loggés ET enregistrés dans le Model Registry ! Vérifie sur : http://localhost:5001")



➡️ Training LogisticRegression avec params: {'C': 1.0, 'solver': 'liblinear'}


2025/03/23 14:21:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/03/23 14:21:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Loan_Model_v1, version 1


🏃 View run fun-mole-252 at: http://localhost:5001/#/experiments/790845414201742872/runs/90478842cc674418b54dde296c9d34ca
🧪 View experiment at: http://localhost:5001/#/experiments/790845414201742872

➡️ Training LogisticRegression avec params: {'C': 0.1, 'solver': 'liblinear'}


2025/03/23 14:21:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/03/23 14:21:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Loan_Model_v1, version 2


Model déjà enregistré : RESOURCE_ALREADY_EXISTS: Registered Model (name=Loan_Model_v1) already exists.
🏃 View run delicate-shark-575 at: http://localhost:5001/#/experiments/790845414201742872/runs/bf9ffb32ad2345d6a054d5f800c6215d
🧪 View experiment at: http://localhost:5001/#/experiments/790845414201742872

➡️ Training LogisticRegression avec params: {'C': 10.0, 'solver': 'lbfgs', 'max_iter': 1000}


2025/03/23 14:21:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/03/23 14:21:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Loan_Model_v1, version 3


Model déjà enregistré : RESOURCE_ALREADY_EXISTS: Registered Model (name=Loan_Model_v1) already exists.
🏃 View run able-yak-472 at: http://localhost:5001/#/experiments/790845414201742872/runs/016e32a0c2be40cbba4c53be942967fb
🧪 View experiment at: http://localhost:5001/#/experiments/790845414201742872
\ Tous les runs sont loggés ET enregistrés dans le Model Registry ! Vérifie sur : http://localhost:5001


In [ ]:
import mlflow.sklearn

model_uri = "models:/Loan_Model_v1/Production"  
model = mlflow.sklearn.load_model(model_uri)
